## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
 
from scipy.stats import norm
from scipy.integrate import quad

import plotly.express as px
import plotly.graph_objects as go

# Distribution plot: https://plotly.com/python/distplot/
import plotly.figure_factory as ff

## Normal approximation to binary game

### Parameters of binary games (winning `b` dollars on prob `p` OR losing 1 dollar on prob `1-p`)

In [2]:
def getbinstat(p, b):
    """Obtain key statistics of binary game."""
    # mean
    miu = round(b * p - (1 - p), 6)
    # SD
    sigma = round((b + 1) * ((p * (1 - p)) ** 0.5), 6)
    # Skewness
    skew = round((p * (b - miu)**3 + (1 - p) * (-1 - miu)**3) / (sigma**3), 6)
    # Excess kurtosis
    kurt = round((p * (b - miu)**4 + (1 - p) * (-1 - miu)**4) / (sigma**4) - 3, 6)
    # Profit Factor
    pf = round(p * b / (1 - p), 6)
    
    
    return miu, sigma, skew, kurt, pf

In [3]:
b1 = 2.0
p1 = 3 / 7
miu1, sig1, skew1, kurt1, pf1 = getbinstat(p1, b1)
print(f'Mean: {miu1}, SD: {sig1}, Skewness: {skew1}, Kurtosis: {kurt1}, Profit Factor: {pf1}')

Mean: 0.285714, SD: 1.484615, Skewness: 0.288676, Kurtosis: -1.916667, Profit Factor: 1.5


### normal moments of a score interval

In [4]:
def getintervalspec(lb, ub, miu=0, sig=1):
    """Obtain the 0th, 1st and 2nd moments of a normal variable between two scores."""
    prob = round(norm.cdf(ub, loc=miu, scale=sig) - norm.cdf(lb, loc=miu, scale=sig), 6)
    func = lambda x, p, m, s: (x ** p) * np.exp(-(x - m)**2 / (2 * s**2)) / (s * (2 * np.pi)**0.5)
    mean = round(quad(func, lb, ub, args=(1, miu, sig))[0], 6) / prob
    sqm = quad(func, lb, ub, args=(2, miu, sig))[0] / prob
    sd = round((sqm - mean ** 2) ** 0.5, 6)
    
    return prob, mean, sd

### estimate of prob and mean return by normal approx

In [5]:
def getnormestmom(lb=0.0, ub=1.0, p=0.4, b=2.0):
    """Obtain estimate of winning prob and mean return by normal extension of binary game."""
    miu, sigma, skew, kurt, pf = getbinstat(p, b) 
    return getintervalspec(lb, ub, miu, sigma)  
    

In [6]:
# Prob of absolute winning
p0 = 0.66
b0 = 0.8242

miu0, sig0, skew0, kurt0, pf0 = getbinstat(p0, b0)
print(f'Parameters of binary games p = {round(p0, 4)}, b = {round(b0, 4)} \n\
Mean: {miu0}, SD: {sig0}, Skewness: {skew0}, Kurtosis: {kurt0}, Profit Factor: {pf0} \n')

# Prob of winning
prob1, mean1, sd1 = getnormestmom(0, b0 * 1000, p0, b0)
print(f'Absolute winning: \nprob: {round(prob1, 4)} \nmean: {round(mean1, 4)} \nSD: {round(sd1, 4)}\n')

# Prob of over 1R return 
prob2, mean2, sd2 = getnormestmom(1, b0 * 1000, p0, b0)
print(f'Winning at least 1R: \nprob: {round(prob2, 4)} \nmean: {round(mean2, 4)} \nSD: {round(sd2, 4)}\n')

# Prob of over-the-odds return
prob3, mean3, sd3 = getnormestmom(b0, b0 * 1000, p0, b0)
print(f'Winning > $b: \nprob: {round(prob3, 4)} \nmean: {round(mean3, 4)} \nSD: {round(sd3, 4)}\n')

# Prob of losing
prob4, mean4, sd4 = getnormestmom(-b0 * 1000, 0, p0, b0)
print(f'Absolute losing: \nprob: {round(prob4, 4)} \nmean: {round(mean4, 4)} \nSD: {round(sd4, 4)}\n')
print(f'Discrete PF: {round(-prob1 * mean1 / (prob4 * mean4), 4)}\n')

# Prob of small P/L
m0 = 0.5
prob5, mean5, sd5 = getnormestmom(-m0, m0, p0, b0)
print(f'P/L within {m0}R: \nprob: {round(prob5, 4)} \nmean: {round(mean5, 4)} \nSD: {round(sd5, 4)}\n')

# Prob of losing 1R
prob6, mean6, sd6 = getnormestmom(-b0 * 1000, -1.0, p0, b0)
print(f'Losing at least 1R: \nprob: {round(prob6, 4)} \nmean: {round(mean6, 4)} \nSD: {round(sd6, 4)}\n')

Parameters of binary games p = 0.66, b = 0.8242 
Mean: 0.203972, SD: 0.86414, Skewness: -0.675519, Kurtosis: -1.543675, Profit Factor: 1.599918 

Absolute winning: 
prob: 0.5933 
mean: 0.7691 
SD: 0.5587

Winning at least 1R: 
prob: 0.1785 
mean: 1.4677 
SD: 0.3946

Winning > $b: 
prob: 0.2365 
mean: 1.3308 
SD: 0.4193

Absolute losing: 
prob: 0.4067 
mean: -0.6204 
SD: 0.4851

Discrete PF: 1.8084

P/L within 0.5R: 
prob: 0.4264 
mean: 0.0217 
SD: 0.2818

Losing at least 1R: 
prob: 0.0818 
mean: -1.3933 
SD: 0.3443



### Tabulate normal approx of different (p,b) with same profit factor

In [7]:
def getnormprofile(pf=1.6, lp=0.32, up=0.68):
    """Obtain a table of notable normal probabilities of binary game normalization."""
    prange = np.arange(lp, up, 0.02)
    df = pd.DataFrame()
    for p in prange:
        b = pf * (1 - p) / p
        df.loc[p, 'b'] = b
        miu, sigma, skew, kurt, pf  = getbinstat(p, b)
        df.loc[p, 'miu'] = miu
        df.loc[p, 'sig'] = sigma
        prob1, mean1, sd1 = getnormestmom(0, b * 1000, p, b)
        df.loc[p, 'win_prob'] = round(prob1, 4)
        df.loc[p, 'win_mean'] = round(mean1, 4)
        prob2, mean2, sd2 = getnormestmom(-b * 1000, 0, p, b)
        df.loc[p, 'loss_prob'] = round(prob2, 4)
        df.loc[p, 'loss_mean'] = round(mean2, 4)
        df.loc[p, 'dPF'] = round(-prob1 * mean1 / (prob2 * mean2), 4)
        prob3, mean3, sd3 = getnormestmom(1, b * 1000, p, b)
        df.loc[p, '+1R_prob'] = round(prob3, 4)
        df.loc[p, '+1R_mean'] = round(mean3, 4)
        prob4, mean4, sd4 = getnormestmom(-b * 1000, -1, p, b)
        df.loc[p, '-1R_prob'] = round(prob4, 4)
        df.loc[p, '-1R_mean'] = round(mean4, 4)        
        prob5, mean5, sd5 = getnormestmom(-0.5, 0.5, p, b)
        df.loc[p, 'small_prob'] = round(prob5, 4)
        df.loc[p, 'small_mean'] = round(mean5, 4)   
        
        
    return df

In [8]:
getnormprofile(pf=1.5)

,b,miu,sig,win_prob,win_mean,loss_prob,loss_mean,dPF,+1R_prob,+1R_mean,-1R_prob,-1R_mean,small_prob,small_mean
0.32,3.187500,0.34,1.953369,0.5691,1.6888,0.4309,-1.4413,1.5475,0.3677,2.3416,0.2464,-2.1600,0.1991,0.0074
0.34,2.911765,0.33,1.853037,0.5707,1.6050,0.4293,-1.3648,1.5632,0.3588,2.2598,0.2365,-2.0864,0.2094,0.0079
0.36,2.666667,0.32,1.760000,0.5721,1.5271,0.4279,-1.2941,1.5779,0.3496,2.1839,0.2266,-2.0187,0.2201,0.0085
0.38,2.447368,0.31,1.673306,0.5735,1.4542,0.4265,-1.2285,1.5916,0.3400,2.1132,0.2168,-1.9559,0.2310,0.0091
0.40,2.250000,0.30,1.592168,0.5747,1.3857,0.4253,-1.1673,1.6043,0.3301,2.0470,0.2071,-1.8975,0.2423,0.0097
0.42,2.071429,0.29,1.515930,0.5759,1.3212,0.4241,-1.1100,1.6160,0.3198,1.9848,0.1974,-1.8431,0.2540,0.0104
0.44,1.909091,0.28,1.444035,0.5769,1.2600,0.4231,-1.0561,1.6266,0.3090,1.9263,0.1877,-1.7921,0.2660,0.0110
0.46,1.760870,0.27,1.376010,0.5778,1.2020,0.4222,-1.0054,1.6361,0.2979,1.8710,0.1780,-1.7442,0.2785,0.0117
0.48,1.625000,0.26,1.311450,0.5786,1.1467,0.4214,-0.9573,1.6445,0.2863,1.8185,0.1683,-1.6991,0.2915,0.0124
0.50,1.500000,0.25,1.250000,0.5793,1.0938,0.4207,-0.9118,1.6517,0.2743,1.7688,0.1587,-1.6564,0.3050,0.0130
